In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 https://developer.download.nvidia.co

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33605939| RGYFDX8QXKEIR|B007KO2MLO|     328837464|           Big Maria|Digital_Ebook_Pur...|          4|            0|          0|   N|                N|              Quirky|Elmore Leonard me...| 2013-09-09|
|         US|   34058393|R13CBGTMNV9R8Z|B005FLODDE|     764276359|The Woman Who Was...|Digital_Ebook_Pur

In [4]:
# Filter total_votes count is equal to or greater than 20
df2 = df.filter(df.total_votes >= 20)
df2.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    4514126|R39ESX43X1SA5T|B006NG0GXQ|     402593554|Forever His: A Ti...|Digital_Ebook_Pur...|          1|           36|         47|   N|                Y|   Completely absurd|From the time tha...| 2013-09-09|
|         US|   45292583|R31QRZ3JAMPBB4|B0078T6YHY|     267532424|A Lady in Defianc...|Digital_Ebook_Pur

In [5]:
# Create a new DF to retrieve all the rows where the number of helpful_votes divided by total_votes >= 50%
df3 = df2.filter((df2.helpful_votes / df2.total_votes) >= 0.5)
df3.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    4514126|R39ESX43X1SA5T|B006NG0GXQ|     402593554|Forever His: A Ti...|Digital_Ebook_Pur...|          1|           36|         47|   N|                Y|   Completely absurd|From the time tha...| 2013-09-09|
|         US|   45292583|R31QRZ3JAMPBB4|B0078T6YHY|     267532424|A Lady in Defianc...|Digital_Ebook_Pur

In [6]:
# Create a new DF to retrieve all the rows where a review written was part of the vine program
paid_df = df3.filter(df3.vine == 'Y')
paid_df.show(10)

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [7]:
# Create a new DF to retrieve all the rows where a review written was not part of the vine program
unpaid_df = df3.filter(df3.vine == "N")
unpaid_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    4514126|R39ESX43X1SA5T|B006NG0GXQ|     402593554|Forever His: A Ti...|Digital_Ebook_Pur...|          1|           36|         47|   N|                Y|   Completely absurd|From the time tha...| 2013-09-09|
|         US|   45292583|R31QRZ3JAMPBB4|B0078T6YHY|     267532424|A Lady in Defianc...|Digital_Ebook_Pur

In [8]:
# Total number of reviews
total_reviews = df.count()
total_reviews

5101693

In [9]:
# Total number of 5-star reviews
total_five_star_reviews = df.filter(df.star_rating == 5).count()
total_five_star_reviews

2952613

In [10]:
# Percentage of five star reviews 
total_percentage = (total_five_star_reviews / total_reviews)*100
total_percentage

57.87516026542562

In [17]:
# Total number of paid reviews
total_paid = paid_df.count()
total_paid

0

In [18]:
# Total number of 5-star reviews
paid_five_star_reviews = paid_df.filter(paid_df.star_rating == 5).count()
paid_five_star_reviews

0

In [19]:
# Percentage of paid five star reviews
paid_five_star_percent = (paid_five_star_reviews / total_paid) * 100
paid_five_star_percent

ZeroDivisionError: ignored

In [20]:
# Total number of unpaid reviews
total_unpaid = unpaid_df.count()
total_unpaid

65149

In [21]:
# Total number of 5-star reviews
unpaid_five_star_reviews = unpaid_df.filter(unpaid_df.star_rating == 5).count()
unpaid_five_star_reviews

24673

In [22]:
# Percentage of unpaid five star reviews
unpaid_five_star_percent = (unpaid_five_star_reviews / total_unpaid) * 100
unpaid_five_star_percent

37.87164806827427